#### DNN 기반 다중분류 모델 구현
- 사용되는 데이터셋 : iris.csv
- feature : 4개
- target : 1개 Setosa와 나머지
- 학습방법 : 지도학습 -> 분류 > 다중분류
- 알고리즘 : 인공신경망(ANN) -> 심층(은닉층) 신경망 -> MLP(층이여러개), DNN(은닉층이 많은 구성) 
- FramWork : Pytorch

In [1]:
# 모듈 로딩
# 모델관련 모듈
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import torch.optim as optim

from torchmetrics.classification import MulticlassF1Score
from torchinfo import summary

# 데이터 전처리 및 시각화 모듈
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# 활용 패키지 버전 체크
def versioncheck():
    print(f' torch  {torch.__version__}')
    print(f' pandas  {pd.__version__}')

versioncheck()

 torch  2.4.1
 pandas  2.0.3


In [3]:
DATA_FILE = r'C:\Users\zizonkjs\머신러닝,딥러닝\data\iris.csv'
irisdf=pd.read_csv(DATA_FILE)
irisdf

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica


In [4]:
labelenco = LabelEncoder()
a=labelenco.fit_transform(irisdf['variety'])
irisdf['variety']= a
irisdf



,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


[2] - 모델 클레스 설계 및 정의 - <hr>
어떤 클래스를 만들까? 고려
클래스 목적 : iris.DataSet 학습 후 추론
클래스 이름 : IrisMCFModel
부모 클래스 : nn.Module
매개 변수   : 층별 입출력 갯수 고정하기 때문에 필요 없음
클래스 속성 : featureDF, targetDF, n_rows, n_features
클래스 기능 : __init__() : 모델 구조, forward() : 순방향 학습 <= 오버라이딩(overriding) 상속관계일 때

클래스 구조  
    -입력층 : 피쳐 4개  퍼셉트론 : 50개(보통 입력 때 많이 주고 갈수록 줄임) (4,10)
    -은닉층 : 입력 10개     출력 5개   (10,5)
    -출력층 : 입력5개      타겟(이진분류) 1개    (5,1)  
  
-손실 함수/ 활성화 함수
    *클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드에 사용
    *함수 형태 ==> torch.nn.functional => forward()메서드에 사용

In [5]:
class IrisMCFModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(4, 10)
        self.hidden_layer=nn.Linear(10, 5)
        self.out_layer=nn.Linear(5, 3)
    
    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y=self.in_layer(input_data) # f1w1+f2w2+f3w3+b 요런 식이 10개(숫자10개)
        y=F.relu(y)                   # 범위 0이상
        
        # 은닉층 : 10개의 숫자 받아오기
        y=self.hidden_layer(y)
        y=F.relu(y)

        # 출력층 : 5개의 숫자 값 => 다중 분류 : 손실함수 CrossEntropyLoss가 내부에서 SoftMax를 처리해줌 (케라스나, 텐서플로우 쓸땐 써야함)
        return self.out_layer(y)

In [6]:
# 모델 인스턴스 생성
model = IrisMCFModel()
print(model)


IrisMCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=3, bias=True)
)


In [7]:
summary(model, input_size=(100,4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisMCFModel                             [100, 3]                  --
├─Linear: 1-1                            [100, 10]                 50
├─Linear: 1-2                            [100, 5]                  55
├─Linear: 1-3                            [100, 3]                  18
Total params: 123
Trainable params: 123
Non-trainable params: 0
Total mult-adds (M): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.02

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐개수 : 3개
- 타겟개수 : 1개
- 클래스이름 : IrisDataSet
- 부모클래스 : utils.data.DataSet
- 속성__필드 : featureDF, targetDF, n_rows, n_featrues  
- 필수 메서드:   
    *__init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정  
    *__len__(self) : 데이터의 개수 반환  
    *__getItem__(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [8]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
        

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        return featureTS, targetTS

In [9]:
# [테스트] 데이터셋 인스턴스 생성
featureDF = irisdf[irisdf.columns[:-1]] # 2D (150,3)
targetDF = irisdf[irisdf.columns[-1:]] # 1D(150,1)


irisDS=IrisDataset(featureDF,targetDF)

# 데이터로더 인스턴스 생성
irisDL = DataLoader(irisDS, batch_size=1)
for feature, label in irisDL:
    print(feature.shape, label.shape, feature, label)
    break

torch.Size([1, 4]) torch.Size([1, 1]) tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[0.]])


[4] 학습 준비
- 학습 횟수 : EPOCH ( 처음부터 끝까지 공부할 횟수 )
- 배치 크기 : BATCH_SIZE(학습량)
- 위치 지정 : DEVICE (텐서 저장 및 실행 위치 GPU/CPU)
- 학 습 률  : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1

In [10]:
# 학습 진행 관련 설정 값
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

- 인스턴스 : 모델, 데이터 셋, 최적화 (, 성능지표) 

In [11]:
# 모델 인스턴스
# 학습용, 검증용, 테스트용 데이터 분리
model = IrisMCFModel()

# 학습용, 검증용, 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')

# 학습용, 검증용, 테스트용 데이터셋 생성
trainDS = IrisDataset(X_train, y_train)
testDS = IrisDataset(X_test, y_test)
valDS = IrisDataset(X_val, y_val)

# 학습용 데이터 로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

# 최적화

(84, 4) (38, 4) (28, 4)
(84, 1) (38, 1) (28, 1)


- 최적화 & 손실함수 인스턴스 생성하기

In [12]:

# 최적화 인스턴스 => w, b model.parameter 전달
optimizer=optim.Adam(model.parameters(), lr=LR)


# 바이너리용 손실함수 인스턴스 => 다중분류
# 예측값을 선형식 결과 값 전달함 ==> AF 처리 안함
crossLoss=nn.CrossEntropyLoss()

[5] 학습진행

In [20]:
len(trainDL), trainDL.__len__()

### models 폴더 아래 프로젝트 폴더 아래 모델 파일을 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris/MCF/'

# 저장 파일명
SAVE_FILE = SAVE_PATH + 'model_train_wbs.pth'

# 모델 구조 및 파라미터 모두 저장 파일명명
SAVE_MODEL=SAVE_PATH+'model_all.pth'

# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더 / 폴더 / ... 하위 폴더까지 생성

In [21]:
# 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY=[[],[]], [[],[]]
CNT = irisDS.n_rows / BATCH_SIZE
print(f'CNT => {CNT}')

# 학습 모니터링 / 스케쥴링 설정
# => LOSS_HISTORY, SCORE_HISTORY 활용
# => 임계기준 : 10번
BREAK_CNT = 0
LIMIT_VALUE = 9
model.train()

for epoch in range(EPOCH):

    # 학습 모드로 모델 설정
    model.train()

    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0,0

    for featureTS, targetTS in trainDL:

        #학습 진행
        pre_y=model(featureTS)

        #손실 계산 : nn.CrossEntropyLoss 요구사항 : 정답/타겟은 0D 또는 1D, 타입은 long
        loss=crossLoss(pre_y, targetTS.reshape(-1).long())
        loss_total += loss.item()


        #성능평가 계산
        score=MulticlassF1Score(num_classes=3)(pre_y, targetTS.reshape(-1))
        score_total += score.item()

        #최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에폭당 검증 기능을 키겠다.
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        
        #평가
        pre_val=model(val_featureTS)

        #손실
        loss_val=crossLoss(pre_val, val_targetTS.reshape(-1).long())

        # 성능평가
        score_val = MulticlassF1Score(num_classes=3)(pre_val, val_targetTS.reshape(-1))

    # 에폭당 손실값과 성능평가 값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_SIZE)
    SCORE_HISTORY[0].append(score_total/BATCH_SIZE)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)
    # # 손실 기준
    if len(LOSS_HISTORY[0]) >= 2:
       if LOSS_HISTORY[1][-1] <= LOSS_HISTORY[1][-2] : BREAK_CNT +=1

    # 성능 기준
    if len(SCORE_HISTORY[1]) == 1: # 첫번째 횟수 저장
       torch.save(model.state_dict(), SAVE_FILE)

       # 모델 전체 저장
       torch.save(model, SAVE_MODEL)
    else:
        if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]) : # 첫번째 점수랑 두번째 점수 비교 후 더 성능이 큰쪽을 저장
            torch.save(model.state_dict(), SAVE_FILE)
            torch.save(model, SAVE_MODEL)

    # 성능이 좋은 학습 가중치 저장

    # 학습중단
    if BREAK_CNT >10:
        print('성능 및 손실 개선이 없어서 학습 중단') 
        break



CNT => 15.0
성능 및 손실 개선이 없어서 학습 중단


In [15]:
print(LOSS_HISTORY)
print(SCORE_HISTORY)

[[1.0515037894248962, 1.020784044265747, 0.9943773806095123, 0.9707448720932007, 0.9494476675987243, 0.9293474376201629, 0.9071545481681824, 0.8827544808387756, 0.8610588490962983, 0.8397171795368195, 0.8179551839828492, 0.7959527850151062, 0.7735464334487915, 0.7504265666007995, 0.7263639807701111, 0.7018619060516358, 0.6773197889328003, 0.6530724108219147, 0.6290122091770172, 0.6052096486091614, 0.5819330632686615, 0.5598554313182831, 0.5383984029293061, 0.5177782654762269, 0.4985188990831375, 0.4804923713207245, 0.46343210339546204, 0.4474304229021072, 0.43213953673839567, 0.41796465814113615, 0.4043865233659744, 0.39125095009803773, 0.3788309782743454, 0.3668397724628448, 0.3552189439535141, 0.34398168325424194, 0.33306857049465177, 0.3225870668888092, 0.3125759452581406, 0.3029536485671997, 0.29361981749534605, 0.2846150740981102, 0.275923889875412, 0.2676573500037193, 0.259690622985363, 0.2520039454102516, 0.24473300129175185, 0.2375758945941925, 0.23105234950780867, 0.2243599563

In [16]:
# 모델 테스트 모드 설정
model.eval()
with torch.no_grad():
    # 검증 데이터셋
    test_featureTS=torch.FloatTensor(testDS.featureDF.values)
    test_targetTS=torch.FloatTensor(testDS.targetDF.values)
        
    #평가
    pre_test=model(test_featureTS)
    print(pre_test)

    #손실
    loss_test=crossLoss(pre_test, test_targetTS.reshape(-1).long())

    # 성능평가
    score_test = MulticlassF1Score(num_classes=3)(pre_test, test_targetTS.reshape(-1))


tensor([[ 18.9187,   5.2703, -18.8482],
        [ -1.9278,   4.2148,  -4.2200],
        [ -7.2223,   4.9612,  -1.9384],
        [ 17.0353,   4.7828, -17.0270],
        [-18.3448,   3.6361,   5.3195],
        [-10.4195,   4.2423,   0.4458],
        [-18.0172,   2.5583,   6.4323],
        [ 13.6307,   4.5253, -14.5874],
        [ 14.1597,   4.1194, -14.3081],
        [-21.2071,   1.9552,   8.7501],
        [ -8.5818,   3.9609,  -0.1405],
        [ 15.6935,   4.6575, -16.0098],
        [-20.8517,   2.0887,   8.4725],
        [ -8.4800,   4.6791,  -1.0274],
        [-10.4382,   3.8449,   0.9597],
        [ 14.4395,   4.1021, -14.5259],
        [ -6.4030,   4.5034,  -1.8473],
        [-11.0613,   3.5591,   1.5430],
        [ 15.2688,   4.4610, -15.3658],
        [ 15.7006,   4.4668, -15.7070],
        [ -9.4698,   3.6928,   0.6326],
        [-11.5397,   3.3477,   2.0101],
        [-13.0852,   3.6622,   2.5997],
        [ 15.7946,   4.4695, -15.8195],
        [-17.9538,   3.0099,   5.8789],
